In [1]:
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\Working_attributes")
from Imports import * 

In [6]:
data = pd.read_excel('2024-10-31_Country_retings_check_m.xlsx')
data_dict = pd.read_excel('rating_values.xlsx', sheet_name='Лист2')

In [27]:
data_dict_work = data_dict.copy()
data_dict

,Rating Category,Description,Value (pts.),Wikirating,S&P Global,Scope,Moody's,Fitch,DBRS
0,NaN,NaN,NaN,long-term,long-term,long-term,long-term,long-term,long-term
1,Prime,An obligor has EXTREMELY STRONG capacity to me...,95 to 100,AAA,AAA,AAA,Aaa,AAA,AAA
2,High grade,An obligor has VERY STRONG capacity to meet it...,90 to less than 95,AA+,AA+,AA+,Aa1,AA+,AA (h)
3,NaN,NaN,85 to less than 90,AA,AA,AA,Aa2,AA,AA
4,NaN,NaN,80 to less than 85,AA-,AA-,AA-,Aa3,AA-,AA (l)
5,Upper Medium grade,An obligor has STRONG capacity to meet its fin...,75 to less than 80,A+,A+,A+,A1,A+,A (h)
6,NaN,NaN,70 to less than 75,A,A,A,A2,A,A
7,NaN,NaN,65 to less than 70,A-,A-,A-,A3,A-,A (l)
8,Lower Medium grade,An obligor has ADEQUATE capacity to meet its f...,60 to less than 65,BBB+,BBB+,BBB+,Baa1,BBB+,BBB (h)
9,NaN,NaN,55 to less than 60,BBB,BBB,BBB,Baa2,BBB,BBB


In [31]:
data_dict_work['Value (pts.)'] = data_dict_work['Value (pts.)'].str.split(' ').str[-1]
data_dict_work = data_dict_work.iloc[1:, [2, 4,6,7]]
data_dict_work

,Value (pts.),S&P Global,Moody's,Fitch
1,100,AAA,Aaa,AAA
2,95,AA+,Aa1,AA+
3,90,AA,Aa2,AA
4,85,AA-,Aa3,AA-
5,80,A+,A1,A+
6,75,A,A2,A
7,70,A-,A3,A-
8,65,BBB+,Baa1,BBB+
9,60,BBB,Baa2,BBB
10,55,BBB-,Baa3,BBB-


In [82]:
data_dict_work_melt = data_dict_work.melt(id_vars='Value (pts.)', value_vars=['S&P Global','Moody\'s','Fitch'])
values_dict = {x:y for x,y in zip(data_dict_work_melt.iloc[:,2].tolist(),data_dict_work_melt.iloc[:,0].tolist())}
values_dict['SD'] = 5
values_dict['Withdrawn'] = np.nan
values_dict['RD'] = 5
values_dict['NR'] = np.nan

In [115]:


# Функция для выбора второго наилучшего рейтинга с учетом нулей
def get_second_highest_rating(row):
    ratings = {'Fitch': row['Fitch_value'], 'Moody\'s': row['Moody\'s_value'], 'S&P': row['S&P_value']}

    # Удаляем рейтинги, равные 0
    ratings = {key: value for key, value in ratings.items() if value != 0}

    # Проверка, если остался только один рейтинг
    if len(ratings) == 1:
        only_value = list(ratings.values())[0]
        only_key = list(ratings.keys())[0]
        # Если единственное значение равно 0, возвращаем NaN и None
        return pd.Series([only_value if only_value != 0 else float('nan'), only_key if only_value != 0 else None],
        index=['второй_наилучший_рейтинг', 'исходный_столбец'])

    # Проверка, если вообще не осталось ненулевых значений
    if len(ratings) < 2:
        return pd.Series([float('nan'), None], index=['второй_наилучший_рейтинг', 'исходный_столбец'])

    # Сортировка оставшихся значений и выбор второго наилучшего
    sorted_ratings = sorted(ratings.items(), key=lambda x: x[1], reverse=True)
    second_best = sorted_ratings[1] # второй наилучший рейтинг
    return pd.Series([second_best[1], second_best[0]], index=['второй_наилучший_рейтинг', 'исходный_столбец'])




# Применение функции к каждой строке DataFrame
data[['второй_наилучший_рейтинг', 'исходный_столбец']] = data.apply(get_second_highest_rating, axis=1)
data

,Code,Countries,Fitch_outlook,Moody's_outlook,S&P_outlook,Fitch,Moody's,S&P,Fitch_date,Moody's_date,S&P_date,Fitch_value,Moody's_value,S&P_value,второй_наилучший_рейтинг,исходный_столбец
0,ML,Mali,NaN,Stable,NaN,Withdrawn,Caa2,NR,2016-08-24 09:50:52,2024-06-12 09:28:24,2021-03-17 16:35:58,0,15,0,15.0,Moody's
1,RU,Russia,NaN,NaN,NaN,Withdrawn,Withdrawn,NR,2022-03-27 16:48:58,2022-03-31 16:36:32,2022-04-11 07:42:47,0,0,0,NaN,NaN
2,BY,Belarus,NaN,Stable,NaN,Withdrawn,C,NR,2023-10-09 09:10:58,2023-06-05 09:55:41,2023-09-11 10:00:02,0,0,0,NaN,NaN
3,AU,Australia,Stable,Stable,Stable,AAA,Aaa,AAA,2023-11-10 08:47:55,2024-10-10 09:12:59,2024-09-26 10:09:05,100,100,100,100.0,Moody's
4,TH,Thailand,Stable,Stable,Stable,BBB+,Baa1,BBB+,2023-11-14 09:51:42,2024-04-12 10:24:00,2023-11-29 09:08:06,65,65,65,65.0,Moody's
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,SV,Salvador,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,0,0,0,NaN,NaN
110,SK,Slovakia,NaN,NaN,NaN,A,A2,A+,2024-06-07 00:00:00,2023-11-24 00:00:00,2024-02-09 00:00:00,75,75,80,75.0,Fitch
111,SI,Slovenia,NaN,NaN,NaN,A,A3,AA-,2024-10-11 00:00:00,2024-10-11 00:00:00,2024-07-05 00:00:00,75,70,85,75.0,Fitch
112,TG,Togo,NaN,NaN,NaN,NaN,B3,B,NaT,2024-09-27 00:00:00,2024-09-27 00:00:00,0,25,30,25.0,Moody's


In [128]:
data_melt_values = data.melt(id_vars=[	'Code','Countries', 'исходный_столбец'], value_vars=['Fitch','Moody\'s', 'S&P'])
data_melt_dates = data.melt(id_vars=[	'Code','Countries', 'исходный_столбец'], value_vars=['Fitch_date','Moody\'s_date', 'S&P_date'])
data_melt_dates['variable'] = data_melt_dates['variable'].str.split('_').str[0]
data_melt_merge = data_melt_values.merge(data_melt_dates, how='left', on=['Code','Countries','variable'])
data_melt_merge = data_melt_merge[data_melt_merge.исходный_столбец_x==data_melt_merge.variable].rename(columns={'variable':'rating_company','value_x':'rating','value_y':'date'})
data_melt_merge = data_melt_merge[['Code','Countries','rating_company','rating','date']]

In [129]:
with pd.ExcelWriter(f'{str(date.today())}_countries_rating.xlsx', engine='openpyxl') as writer:
    data_melt_merge.to_excel(writer, index=False, sheet_name='one_rating')
    data.to_excel(writer, index=False, sheet_name='full_data')